# Finding, downloading, and processing relevant X-ray observations

We want to examine the X-ray emission of these groups, some of which we know have had targeted XMM observations. The XGA module doesn't support telescopes other than XMM at the moment, but we may download other available observations for further context/external analysis.

Once identified, the XMM observations will be downloaded, and their raw data processed into a state suitable for analysis.

## Import statements

In [1]:
import daxa
daxa.NUM_CORES = 15
daxa.OUTPUT = "/mnt/gs21/scratch/turne540/OVI_groups/data/"
daxa.config.OUTPUT = "/mnt/gs21/scratch/turne540/OVI_groups/data/"
daxa.mission.base.OUTPUT = "/mnt/gs21/scratch/turne540/OVI_groups/data/"
from daxa.archive import Archive
from daxa.process.simple import full_process_xmm
from daxa.mission import XMMPointed, Chandra, NuSTARPointed, ROSATPointed, ROSATAllSky
from daxa.exceptions import NoObsAfterFilterError

import pandas as pd
from astropy.units import Quantity
import os
from warnings import warn

## Reading the sample file

Reading in the sample file that contains the positions of the groups, these will be used to locate relevant X-ray observations:

In [2]:
samp = pd.read_csv("../sample_files/init_group_info.csv")
samp

,name,est_ra,est_dec,redshift
0,25124,243.629055,26.730240,0.186
1,44739,229.793620,28.331750,0.118
2,19670A,150.256710,50.793942,0.134
3,19670B,150.214920,50.805014,0.134
4,12833,129.496800,44.248700,0.145
5,44858,230.112000,28.887750,0.127
6,PHL1811,328.769920,-9.588805,0.077
7,50433,157.732000,30.844000,0.136


## Searching for data

We can use DAXA classes to search the online databases of observations maintained for several X-ray telescopes. We will only make use of XMM data in this analysis, but it will be interesting to see what other data are available.

### XMM-Newton Pointed

We define an XMMPointed instance, which makes the distinction between XMM data that were taken in 'pointing' mode, when the attitude of the spacecraft was fixed (or nearly fixed), and data taken when XMM was slewing to its next target. Slew data is harder to analyse, with decreased spatial resolution, and is not yet implemented in DAXA.

The RA-Dec values for the groups of interest are used to filter the entire archive on position, searching for any XMM pointed observations with an aimpoint within the approximate field-of-view of the telescope:

In [3]:
# Define the instance
xmm = XMMPointed()
# This uses the pre-defined XMM field of view to search for observations around 
#  the positions of our groups - the same search will be performed using other telescopes
xmm.filter_on_positions(samp[['est_ra', 'est_dec']].values)

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/xmm.py:80: UserWarning: 140 of the 17116 observations located for this mission have been removed due to NaN RA or Dec values
  self._fetch_obs_info()


This part is a small bodge to account for the fact that the proprietary data I want to make use of in this notebook isn't associated with my XMM account, and I've just been handed the downloaded data. This part may not last very long as I think I'll probably add a more elegant way of manually adding data to a DAXA mission (one that doesn't involve accessing protected attributes). 

This essentially relies on us having copied the unpacked ODFs into the xmm_pointed_raw DAXA directory, and then manually altering the observation info dataframe to trick DAXA that it is in a proprietary usable period. It'll check for the ObsIDs when it does the download, find them there, and that will be that:

In [4]:
prop_obsids = ['0900700101', '0900700201']
for obs_id in prop_obsids:
    if os.path.exists('../data/xmm_pointed_raw/{}'.format(obs_id)): 
        rel_rows_msk = xmm._obs_info['ObsID'] == obs_id
        xmm._obs_info.loc[rel_rows_msk, 'proprietary_usable'] = True
xmm.filtered_obs_info

,ra,dec,ObsID,start,science_usable,duration,proprietary_end_date,revolution,proprietary_usable,end
917,157.746300,31.048890,0102040301,2000-12-06 20:46:26,True,0 days 08:08:31,2002-04-06 00:00:00,182,True,2000-12-07 04:54:57
922,157.746300,31.048890,0102041001,2000-12-07 04:57:14,True,0 days 01:30:06,2002-04-06 00:00:00,182,True,2000-12-07 06:27:20
3802,328.756200,-9.373528,0204310101,2004-11-01 09:06:42,True,0 days 09:08:39,2005-12-01 00:00:00,897,True,2004-11-01 18:15:21
12006,328.756250,-9.373333,0761910201,2015-11-29 09:38:07,True,0 days 16:30:00,2016-12-11 23:00:00,2925,True,2015-11-30 02:08:07
13847,230.111667,28.886111,0820240301,2019-01-14 04:17:53,True,0 days 06:06:40,2020-02-04 23:00:00,3498,True,2019-01-14 10:24:33
14961,129.414208,44.284083,0861080201,2020-10-07 12:28:01,True,0 days 06:18:20,2021-11-13 00:00:00,3815,True,2020-10-07 18:46:21
14962,129.414208,44.284083,0861080501,2020-10-07 10:09:04,True,0 days 02:18:57,2021-11-13 00:00:00,3815,True,2020-10-07 12:28:01
14965,150.166375,50.783694,0861080101,2020-10-11 11:57:49,True,0 days 06:23:20,2021-11-02 00:00:00,3817,True,2020-10-11 18:21:09
14972,150.166375,50.783694,0861080601,2020-10-11 09:53:44,True,0 days 02:04:05,2021-11-02 00:00:00,3817,True,2020-10-11 11:57:49
16245,229.803958,28.380667,0900700201,2022-08-17 12:04:49,True,0 days 08:13:20,2023-08-30 00:00:00,4155,True,2022-08-17 20:18:09


### Chandra

We perform a similar search in the Chandra archive - be aware that the odd geometry of some of Chandra's detectors makes the search radius defined in the DAXA class quite approximate:

In [5]:
chandra = Chandra()
chandra.filter_on_positions(samp[['est_ra', 'est_dec']].values)
chandra.filtered_obs_info

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/mission/base.py:781: UserWarning: Chandra FoV are difficult to define, as they can be strongly dependant on observation mode; as such takethese as very approximate.
  fov = self.fov


,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,instrument,grating,data_mode
8973,129.49708,44.24889,15378,True,True,2013-01-04 02:14:34.000002,2013-01-04 07:49:14.000002,0 days 05:34:40,2014-01-04,GCL,ACIS-I,NONE,TE_00458
9507,130.03917,44.36500,21564,True,True,2019-01-04 18:55:19.999998,2019-01-05 00:24:59.999998,0 days 05:29:40,2020-01-07,GCL,ACIS-I,NONE,TE_006E6
12236,130.03917,44.36500,22035,True,True,2019-01-06 19:19:28.000001,2019-01-06 22:40:48.000001,0 days 03:21:20,2020-01-07,GCL,ACIS-I,NONE,TE_006E6
14098,328.75625,-9.37353,2958,True,True,2001-12-17 22:59:37.999999,2001-12-18 01:45:37.999999,0 days 02:46:00,2002-12-21,AGN,ACIS-S,NONE,TE_002A2
14568,328.75625,-9.37353,2957,True,True,2001-12-05 08:43:41.000002,2001-12-05 11:22:01.000002,0 days 02:38:20,2002-12-18,AGN,ACIS-S,NONE,TE_002A2
18016,230.48917,28.98831,4791,True,True,2004-04-12 03:22:08.000002,2004-04-12 04:39:28.000002,0 days 01:17:20,2005-04-20,AGN,ACIS-S,NONE,TE_003C2
19969,328.75625,-9.37342,15357,True,True,2012-11-24 20:01:44.999999,2012-11-24 20:36:34.999999,0 days 00:34:50,2013-11-26,AGN,ACIS-S,NONE,TE_00B4A


We then download the Chandra data in their standard format, which will have pre-generated images that can be examined for emission from the groups we are interested in:

In [6]:
chandra.download(download_standard=True)

### NuSTAR Pointed

We search the NuSTAR archive, though the small field-of-view of NuSTAR means that we do not find any serindipitious data (a try-except is used so that it doesn't halt the notebook):

In [7]:
nustar = NuSTARPointed()
try:
    nustar.filter_on_positions(samp[['est_ra', 'est_dec']].values)
    nustar.filtered_obs_info
except NoObsAfterFilterError:
    warn("No NuSTAR data for our groups are found!", stacklevel=2)

/mnt/home/turne540/software/anaconda3/envs/test_daxa_dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3460: UserWarning: No NuSTAR data for our groups are found!
  exec(code_obj, self.user_global_ns, self.user_ns)


## ROSAT Data

We can also search the ROSAT archive, both the all-sky and the pointed observations, using DAXA missions. We do not necessarily expect to be able to see our groups in these data, but we might as well look.

### ROSAT All Sky Survey

These 'observations' are 6x6 degree chunks of the sky from the all sky survey, with low exposure times and poor spatial resolution when compared to some of the pointed observations made by ROSAT; each of our groups should be in RASS (it is an all-sky survey after all):

In [8]:
rs = ROSATAllSky()
rs.filter_on_positions(samp[['est_ra', 'est_dec']].values)
rs.filtered_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category
241,245.17096,28.125,RS931240N00,True,1990-07-30,1991-08-13,0 days 00:12:16,ASK
369,226.55017,28.125,RS931237N00,True,1990-12-30,1991-08-13,0 days 00:10:27,ASK
437,154.88338,50.625,RS930819N00,True,1990-10-17,1990-11-11,0 days 00:09:54,ASK
471,146.50867,50.625,RS930818N00,True,1990-10-10,1990-11-06,0 days 00:09:35,ASK
629,131.25000,45.000,RS930918N00,True,1990-10-03,1990-10-27,0 days 00:08:23,ASK
753,158.27508,28.125,RS931226N00,True,1990-11-02,1990-11-23,0 days 00:07:51,ASK
1053,232.75833,28.125,RS931238N00,True,1990-07-30,1991-08-13,0 days 00:06:55,ASK
1195,329.06250,-11.250,RS931959N00,True,1990-10-31,1990-11-19,0 days 00:06:09,ASK


We download the relevant RASS data with `download_products=True`, which will acquire images and exposure maps as well as event lists:

In [9]:
rs.download(download_products=True)

### ROSAT Pointed

The pointed phase could provide data from either the PSPC instrument (this type of instrument was used for the all-sky survey, and has a very large field of view), or the HRI (much higher spatial resolution, but essentially non-existent energy resolution). 

The search method uses appropriate field-of-view values for the different instruments to search for data:

In [10]:
rp = ROSATPointed()
rp.filter_on_positions(samp[['est_ra', 'est_dec']].values)
rp.filtered_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,instrument,with_filter,target_category,target_name,proc_rev,fits_type
1402,157.75,31.05,RH800762N00,True,1996-05-11 12:00:40.999997,1996-05-27 17:24:16.000001,0 days 05:12:54,HRI,N,GCL,B2 1028+313,2,RDF 3_4
2657,150.06,50.62,RH701869N00,True,1995-05-17 23:58:44.999996,1995-05-19 00:43:29.999997,0 days 02:50:32,HRI,N,AGN,RX J10002+5036,2,RDF 3_4
2771,158.75,30.68,RP800350N00,True,1993-05-28 12:25:17.999996,1993-06-02 22:01:13.999996,0 days 02:43:51,PSPCB,N,GCL,A1045,2,RDF 3_4
3671,157.75,31.05,RF700433N00,True,1992-05-02 14:17:05.999997,1992-05-02 19:33:46.999999,0 days 01:54:48,PSPCB,Y,AGN,B2 1028+313,2,RDF 3_4
6954,157.75,31.05,RP700433N00,True,1992-05-01 19:08:26.999998,1992-05-02 19:01:36.000002,0 days 00:40:42,PSPCB,N,AGN,B2 1028+313,2,RDF 3_4


Again we download with `download_products=True`, though HRI observations do not have exposure maps in the archive to be downloaded, we are not sure why:

In [11]:
rp.download(download_products=True)

## Creating an Archive

We'll set up a DAXA archive - which is be most useful when processing multi-mission data, but here will just contain XMM data, as we can't yet process any of the other telescopes with DAXA. The archive will be named, and the processed raw data will be stored in it, as well as logs of the processing steps.

It also automatically downloads the XMM data, if we haven't already triggered that from the mission class instance:

In [12]:
ovi_group_arch = Archive(xmm, 'OVIGroups', clobber=True)
ovi_group_arch.info()

/mnt/ufs18/home-218/turne540/code/DAXA/daxa/archive/base.py:101: UserWarning: The number of cores assigned to XMMPointed downloads has been capped at 10, this will minimise dropped connections.
  mission.download()


-----------------------------------------------------
Number of missions - 1
Total number of observations - 11
Beginning of earliest observation - 2000-12-06 20:46:26
End of latest observation - 2022-09-12 10:34:47

-- XMM-Newton Pointed --
   Internal DAXA name - xmm_pointed
   Chosen instruments - M1, M2, PN
   Number of observations - 11
   Fully Processed - False
-----------------------------------------------------



## Processing XMM data

The different processing steps that need to be applied to XMM can be controlled separately, with an array of user-configurable options; but a full processing stack to go from raw data to cleaned event lists, images, and exposure maps is available. It uses default settings and should produce very useful data:

In [13]:
full_process_xmm(ovi_group_arch)

XMM-Newton Pointed - Generating calibration files: 100%|██████████| 11/11 [01:31<00:00,  8.35s/it]
XMM-Newton Pointed - Generating ODF summary files: 100%|██████████| 11/11 [00:15<00:00,  1.42s/it]
XMM-Newton Pointed - Assembling PN and PN-OOT event lists: 100%|██████████| 8/8 [16:16<00:00, 122.10s/it]
XMM-Newton Pointed - Assembling MOS event lists: 100%|██████████| 20/20 [02:03<00:00,  6.17s/it]
XMM-Newton Pointed - Finding PN/MOS soft-proton flares: 100%|██████████| 28/28 [00:44<00:00,  1.58s/it]
XMM-Newton Pointed - Generating cleaned PN/MOS event lists: 100%|██████████| 28/28 [00:12<00:00,  2.27it/s]
XMM-Newton Pointed - Generating final PN/MOS event lists: 100%|██████████| 24/24 [00:00<00:00, 40.70it/s]
Generating products of type(s) expmap: 100%|██████████| 24/24 [04:14<00:00, 10.60s/it]


## Checking the data

Here we use the logging facilities of DAXA archives to investigate whether all the data were processed fully and, if they weren't, which observations failed and at which steps. That way we can decide whether there is something extra that can be done for those observations, or there are no further steps to take.

### Overall processing success

The first step is to check the 'overall' success, which will only report as False if all data for an entire observation (i.e. PN, MOS1, and MOS2) have been marked as failing processing. We can see that the processing of two observations has failed outright:

In [14]:
ovi_group_arch.final_process_success

{'xmm_pointed': {'0102040301': True,
  '0102041001': False,
  '0204310101': True,
  '0761910201': True,
  '0820240301': True,
  '0861080201': True,
  '0861080501': False,
  '0861080101': True,
  '0861080601': False,
  '0900700201': True,
  '0900700101': True}}

### 0102041001 - Outright failure

We can examine the 'observation summary' for 0102041001, which is parsed from the SAS summary file generated by ODF ingest. While the instruments all took science observations, we can see that each of the three telescopes had their filter wheels in 'CalClosed' position:

In [20]:
ovi_group_arch.observation_summaries['xmm_pointed']['0102041001']

{'M1': {'active': True,
  'num_exp': 1,
  'exposures': {'S004': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimePartialW3',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging'}}}},
 'M2': {'active': True,
  'num_exp': 1,
  'exposures': {'S006': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimePartialW3',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging'}}}},
 'PN': {'active': True,
  'num_exp': 1,
  'exposures': {'S003': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeSmallWindow',
    'filter': 'CalClosed',
    'ccd_modes': {4: 'Imaging'}}}}}

### 0861080601 - Outright failure

We can examine the 'observation summary' for 0861080601, which is parsed from the SAS summary file generated by ODF ingest. While the instruments all took science observations, we can see that each of the three telescopes had their filter wheels in 'CalClosed' position. Given the similar ObsID of the other outright failure, it seems likely that it will be for the same reason:

In [15]:
ovi_group_arch.observation_summaries['xmm_pointed']['0861080601']

{'M1': {'active': True,
  'num_exp': 1,
  'exposures': {'S001': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     7: 'Imaging'}}}},
 'M2': {'active': True,
  'num_exp': 1,
  'exposures': {'S002': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging'}}}},
 'PN': {'active': True,
  'num_exp': 14,
  'exposures': {'S003': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging',
     8: 'Imaging',
     9: 'Imaging',
     10: 'Imaging',
     11: 'Imaging',
     12: 'Imaging'

### 0861080501 - Outright failure

This has failed outright for the same reason as 0861080601, all observations were taken with the CalClosed filter:

In [16]:
ovi_group_arch.observation_summaries['xmm_pointed']['0861080501']

{'M1': {'active': True,
  'num_exp': 1,
  'exposures': {'S001': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     7: 'Imaging'}}}},
 'M2': {'active': True,
  'num_exp': 1,
  'exposures': {'S002': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging'}}}},
 'PN': {'active': True,
  'num_exp': 14,
  'exposures': {'S003': {'scheduled': True,
    'type': 'SCIENCE',
    'mode': 'PrimeFullWindow',
    'filter': 'CalClosed',
    'ccd_modes': {1: 'Imaging',
     2: 'Imaging',
     3: 'Imaging',
     4: 'Imaging',
     5: 'Imaging',
     6: 'Imaging',
     7: 'Imaging',
     8: 'Imaging',
     9: 'Imaging',
     10: 'Imaging',
     11: 'Imaging',
     12: 'Imaging'

### Tracing ObsID-Instrument-SubExposure failures at different stages

We can also check the success flags for the different processing stages at an individual sub-exposure level, to see whether any of the observations had a partial failure in processing. The only problematic sub-exposure we find is '0801892301PNS003', the scheduled PN observation of 0801892301, which failed the `espfilt` (soft-proton filtering) stage and was not allowed to continue:

In [17]:
ovi_group_arch.process_success

{'xmm_pointed': {'cif_build': {'0820240301': True,
   '0861080101': True,
   '0102041001': True,
   '0102040301': True,
   '0900700101': True,
   '0861080601': True,
   '0900700201': True,
   '0204310101': True,
   '0861080501': True,
   '0761910201': True,
   '0861080201': True},
  'odf_ingest': {'0102041001': True,
   '0861080601': True,
   '0861080501': True,
   '0861080101': True,
   '0820240301': True,
   '0102040301': True,
   '0861080201': True,
   '0204310101': True,
   '0761910201': True,
   '0900700101': True,
   '0900700201': True},
  'epchain': {'0102040301PNS003': True,
   '0820240301PNS003': True,
   '0861080101PNS003': True,
   '0861080201PNS003': True,
   '0900700101PNS003': True,
   '0204310101PNS003': True,
   '0900700201PNS003': True,
   '0761910201PNS003': True},
  'emchain': {'0861080201M1S001': True,
   '0861080201M2S002': True,
   '0900700201M1U003': True,
   '0102040301M1S004': True,
   '0861080201M1U003': True,
   '0900700101M1S001': True,
   '0861080101M1S001'

## Assigning regions to XMM observations

Source regions are a key part of any analysis, but DAXA does not currently have the capability to apply source detection to observations that it processes. As such we shall use source regions generated by the XMM Cluster Survey's (XCS) XAPA source finder. Passing them into the DAXA archive will store them in the DAXA archive directory structure:

In [18]:
reg_path = "../sample_files/xcs_regions/{o_id}_sky_regions.reg"

In [19]:
av_reg = [f.split('_')[0] for f in os.listdir('../sample_files/xcs_regions/')]
reg_dec = {'xmm_pointed': {o: {'regions': reg_path.format(o_id=o)} for o in av_reg}}

ovi_group_arch.source_regions = reg_dec